In [26]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [27]:
weather_data = pd.read_csv("Resources/Weather Data/GlobalLandTemperaturesByState.csv")

weather_data.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,State,Country
0,1855-05-01,25.544,1.171,Acre,Brazil
1,1855-06-01,24.228,1.103,Acre,Brazil
2,1855-07-01,24.371,1.044,Acre,Brazil
3,1855-08-01,25.427,1.073,Acre,Brazil
4,1855-09-01,25.675,1.014,Acre,Brazil


In [28]:
# Drop unnecessary columns
weather_data = weather_data.drop(["AverageTemperatureUncertainty"], axis=1)

In [29]:
# change the dt column to datetime
weather_data["dt"] = pd.to_datetime(weather_data["dt"], format="%Y-%M-%d")
weather_data["year"] = weather_data["dt"].dt.strftime("%Y")

weather_data

,dt,AverageTemperature,State,Country,year
0,1855-01-01 00:05:00,25.544,Acre,Brazil,1855
1,1855-01-01 00:06:00,24.228,Acre,Brazil,1855
2,1855-01-01 00:07:00,24.371,Acre,Brazil,1855
3,1855-01-01 00:08:00,25.427,Acre,Brazil,1855
4,1855-01-01 00:09:00,25.675,Acre,Brazil,1855
...,...,...,...,...,...
645670,2013-01-01 00:05:00,21.634,Zhejiang,China,2013
645671,2013-01-01 00:06:00,24.679,Zhejiang,China,2013
645672,2013-01-01 00:07:00,29.272,Zhejiang,China,2013
645673,2013-01-01 00:08:00,29.202,Zhejiang,China,2013


In [30]:
#only selecting data from the United States and from 1875 and on
us_weather = weather_data.loc[weather_data["Country"] == "United States"]
us_weather = us_weather.loc[(us_weather["dt"] > "1875-01-01") & (us_weather["dt"] < "2013-01-01")]

In [31]:
#selecting only the data from california, florida, and illinois
ca_weather = us_weather.loc[us_weather["State"] == "California"].copy()
fl_weather = us_weather.loc[us_weather["State"] == "Florida"].copy()
il_weather = us_weather.loc[us_weather["State"] == "Illinois"].copy()

ca_weather

,dt,AverageTemperature,State,Country,year
71370,1875-01-01 00:01:00,5.000,California,United States,1875
71371,1875-01-01 00:02:00,7.469,California,United States,1875
71372,1875-01-01 00:03:00,8.403,California,United States,1875
71373,1875-01-01 00:04:00,13.922,California,United States,1875
71374,1875-01-01 00:05:00,18.116,California,United States,1875
...,...,...,...,...,...
73021,2012-01-01 00:08:00,25.407,California,United States,2012
73022,2012-01-01 00:09:00,22.760,California,United States,2012
73023,2012-01-01 00:10:00,16.638,California,United States,2012
73024,2012-01-01 00:11:00,11.475,California,United States,2012


In [32]:
#concatenating the data together into one dataframe
frames = [ca_weather, fl_weather, il_weather]

state_weather = pd.concat(frames)

In [33]:
# adding a column for the Average Temperature in Degrees Fahrenheit
state_weather["Average Temperature (F)"] = (state_weather["AverageTemperature"] * (9 / 5)) + 32

# converting float type columns to integers and rounding to the nearest whole number
state_weather['AverageTemperature'] = state_weather['AverageTemperature'].astype(int)
state_weather['Average Temperature (F)'] = state_weather['Average Temperature (F)'].astype(int)
state_weather['year'] = state_weather['year'].astype(int)

state_weather

,dt,AverageTemperature,State,Country,year,Average Temperature (F)
71370,1875-01-01 00:01:00,5,California,United States,1875,41
71371,1875-01-01 00:02:00,7,California,United States,1875,45
71372,1875-01-01 00:03:00,8,California,United States,1875,47
71373,1875-01-01 00:04:00,13,California,United States,1875,57
71374,1875-01-01 00:05:00,18,California,United States,1875,64
...,...,...,...,...,...,...
173507,2012-01-01 00:08:00,23,Illinois,United States,2012,74
173508,2012-01-01 00:09:00,18,Illinois,United States,2012,65
173509,2012-01-01 00:10:00,11,Illinois,United States,2012,53
173510,2012-01-01 00:11:00,5,Illinois,United States,2012,41


In [34]:
#Rename Column Headers
state_weather.rename(columns={"dt": "Date", "AverageTemperature":"Average Temperature (C)", "year": "Year"})
state_weather.dtypes

dt                         datetime64[ns]
AverageTemperature                  int32
State                              object
Country                            object
year                                int32
Average Temperature (F)             int32
dtype: object

In [35]:
# # Combine columns based on common values
state_weather_year = state_weather.groupby(["State", "year"]).sum()

state_weather_year = state_weather_year.rename(columns={"AverageTemperature": "Total (C)", "Average Temperature (F)": "Total (F)"})

In [36]:
state_weather_year["Average Temperature (F)"] = (state_weather_year["Total (F)"] / 12).round(decimals=2)
state_weather_year["Average Temperature (C)"] = (state_weather_year["Total (C)"] / 12).round(decimals=2)

state_weather_year

Total (C)  Total (F)  Average Temperature (F)  \
State      year                                                  
California 1875        173        699                    58.25   
           1876        165        684                    57.00   
           1877        170        696                    58.00   
           1878        165        683                    56.92   
           1879        162        681                    56.75   
...                    ...        ...                      ...   
Illinois   2008        128        611                    50.92   
           2009        128        617                    51.42   
           2010        139        636                    53.00   
           2011        144        638                    53.17   
           2012        156        670                    55.83   

                 Average Temperature (C)  
State      year                           
California 1875                    14.42  
           1876                    13.75  
           1877                    14.17  
           1878                    13.75  
           1879                    13.50  
...                                  ...  
Illinois   2008                    10.67  
           2009                    10.67  
           2010                    11.58  
           2011                    12.00  
           2012                    13.00  

[414 rows x 4 columns]

In [37]:
state_weather.to_csv("Resources/Cleaned Weather Data/StateWeatherCleaned.csv")
state_weather_year.to_csv("Resources/Cleaned Weather Data/StateWeatherByYearCleaned.csv")

In [38]:
# adding a column for the Average Temperature in Degrees Fahrenheit to Cal, Illinois and Florida
ca_weather["Average Temperature (F)"] = (ca_weather["AverageTemperature"] * (9 / 5)) + 32
il_weather["Average Temperature (F)"] = (il_weather["AverageTemperature"] * (9 / 5)) + 32
fl_weather["Average Temperature (F)"] = (fl_weather["AverageTemperature"] * (9 / 5)) + 32

# converting float type columns to integers and rounding to the nearest whole number
ca_weather['AverageTemperature'] = ca_weather['AverageTemperature'].astype(int)
ca_weather['Average Temperature (F)'] = ca_weather['Average Temperature (F)'].astype(int)
il_weather['AverageTemperature'] = il_weather['AverageTemperature'].astype(int)
il_weather['Average Temperature (F)'] = il_weather['Average Temperature (F)'].astype(int)
fl_weather['AverageTemperature'] = fl_weather['AverageTemperature'].astype(int)
fl_weather['Average Temperature (F)'] = fl_weather['Average Temperature (F)'].astype(int)

ca_weather.dtypes

dt                         datetime64[ns]
AverageTemperature                  int32
State                              object
Country                            object
year                               object
Average Temperature (F)             int32
dtype: object

In [48]:
#Rename Column Headers
ca_weather = ca_weather.rename(columns={"dt": "Date"})
il_weather = il_weather.rename(columns={"dt": "Date"})
fl_weather = fl_weather.rename(columns={"dt": "Date"})

In [49]:
ca_ml = ca_weather.groupby("year").mean().round(decimals=2)
fl_ml = fl_weather.groupby("year").mean().round(decimals=2)
il_ml = il_weather.groupby("year").mean().round(decimals=2)
ca_ml

,Average Temperature (C),Average Temperature (F)
year,,
1875,14.42,58.25
1876,13.75,57.00
1877,14.17,58.00
1878,13.75,56.92
1879,13.50,56.75
...,...,...
2008,14.50,58.75
2009,14.50,58.33
2010,13.83,57.50


In [50]:
ca_weather.to_csv("Resources/Cleaned Weather Data/CAWeatherCleaned.csv")
il_weather.to_csv("Resources/Cleaned Weather Data/ILWeatherCleaned.csv")
fl_weather.to_csv("Resources/Cleaned Weather Data/FLWeatherCleaned.csv")
ca_ml.to_csv("Resources/Cleaned Weather Data/CAMachineLCleaned.csv")
il_ml.to_csv("Resources/Cleaned Weather Data/ILMachineLCleaned.csv")
fl_ml.to_csv("Resources/Cleaned Weather Data/FLMachineLCleaned.csv")